# Movie Recommendation System
Im Folgenden wird ausführlich erläutert, wie im Rahmen der Vorlesung 'Machine Learning' ein Movie-Recommendation-System entworfen worden ist, aber auch welche Ansätze fehlgeschlagen und welche Fehler und Probleme aufgetreten sind.
### Datensätze
First things first: Um eine Grundlage für ein System zu schaffen, welches Vorschläge bezüglich Filmen schaffen soll, bedarf es einer gewissen Menge an Daten. Es folgt ein kleiner Überblick zu den verfügbaren Daten, aus welchen dann die relevanten ausgewählt worden sind.

| Movies | Credits | Ratings |
| --- | --- | --- |
| Movie ID | Movie ID | Movie ID |
| Title | Cast | User ID |
| Genres | Crew | Rating |
| Year |  | Timestamp |
| Languages |
| Popularity |
| Runtime |
| Vote Count (IMDB) |
| Vote Average |


<br>
<br>

### Die erste Idee

Ein bekannter Ansatz bei Recommendation-Systemen ist das sogenannte __Collaborative-filtering__. Grundlegend basiert dieses darauf, dann Nutzer, welche in der Vergangenheit ähnliche Muster aufgewiesen, diese auch in der Zukunft aufweisen werden.<br>
In unserem Fall könnten wir also für einen gegebenen Nutzer, passende zugehörige Nutzer finden, welche in der Vergangenheit die gleichen Filme ähnlich bewertet haben. Problematisch ist jedoch, dass der Geschmack der Nutzer sich im Verlauf der Zeit jedoch ändern und somit unser Ergebnis verfälschen. Anstelle dessen werden wir im folgenden die Ähnlichkeit eines Films mit Filmen, die von einem Nutzer bereits bewertet hat feststellen und anhand dieser eine Vorhersage abgeben. <br>
Methoden zum Bestimmen der Ähnlichkeit zweier Filme sind beispielsweise die __Pearson-Korrelation__ oder __Kosinus-Ähnlichkeit__.<br>
Ein bekanntes Probem hierbei stellt jedoch der Mangel an Skalierbarkeit und Sparsamkeit dar, da der Rechenaufwand mit Nutzern und auch Filmen steigen wird. Um dieses Problem zu lösen, wird ein latentes Variablenmodell genutzt um den Zusammenhang zwischen Nutzern und Filmen festzustellen. Anders ausgedrückt: Wir konzentrieren uns auf einen zu betrachtenden Faktor und bringen Filme und Nutzer auf die selbe Dimension um so den Zusammenhang dieser besser zu verstehen. Dies wird auch __Singular Value Decomposition__. Dadurch machen wir aus unserem Recommendation-Problem ein Optimierungs-Problem machen und können mit dem __RMSE__ eine Bewertung aufstellen.

# Datenimport
Anhand der eben beschriebenen Idee, stellt man fest, dass für diese Art eines Modells nur die Rating-Daten nötig sind, denn außer der Movie-ID muss der Computer nichts über den Film selber wissen.

In [12]:
ratings = pd.read_csv('movie_data/ratings_small.csv')

# Cross Validation
Die Library __surprise__ gibt uns die nötige Funktion zur Singular Value Decomposition und einen Reader, welcher dazu dient Ratings zu konvertieren. Ebenfalls gibt es und die Funktion Datasets aus Pandas Dataframes zu erstellen.

In [15]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd = SVD()
cross_validate(svd, data)

{'test_rmse': array([0.89122487, 0.89993953, 0.89916626, 0.89369484, 0.89863653]),
 'test_mae': array([0.68673109, 0.69253091, 0.69356544, 0.68931726, 0.69205333]),
 'fit_time': (6.678909063339233,
  6.579374074935913,
  6.739435195922852,
  6.6382896900177,
  6.335058927536011),
 'test_time': (0.1655576229095459,
  0.17256999015808105,
  0.16954851150512695,
  0.17253923416137695,
  0.18454408645629883)}

Anhand dieser Ausgabe können wir erkennen, dass wir einen durchschnittlichen RMSE von 0.89 haben, was sich als sehr gut erweist. Als nächstes wollen wir nun endlich ein Modell trainieren und Vorhersagen treffen. <br>
Auch hierbei kommt und die __surprise__ library zu gunsten. Über die Methode build_full_trainset können wir aus unserem eben definierten Dataset ein Trainingset definieren. Anhand dieses Trainingssatz können wir mit svd-Modell nun auch Vorhersagen treffen.

In [20]:
trainset = data.build_full_trainset()
svd.fit(trainset)
# Vorhersage für den Nutzer mit der ID 1 und für den Film mit der ID 302
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.6617475877384873, details={'was_impossible': False})

__est__ beschreibt hier also die erwartete Bewertung. Nicht schlecht!<br>
### Die Alternative
An diesem Punkt haben wir uns als Gruppe jedoch entschieden, dass diese Alternative nicht ganz unserer Idee entspricht. Wir wollen etwas mehr Praxis in dieses Projekt bringen und auch in der Lage sein, Vorschläge für uns selber bekommen zu können, nicht nur für IDs in einem Datensatz. <br>
#### Die Idee
Eine App in welcher man mehrere Filme auswählen kann und anhand dieser Auswahl Vorschläge bekommt.
#### Der Ansatz
Neben dem Collaborative-Filtering ist einer der typischen Ansätze bei Recommendation-Systemen das Content-based-Filtering. Hierbei werden Ähnlichkeiten zwischen Filmen direkt gesucht und dann als Vorschlag geliefert. Es werden so zwar keine Informationen über den Nutzer einbezogen, jedoch kann dadurch ausgeglichen werden, dass der Nutzer sich ja Filme aussuchen kann, für welche er Vorschläge bekommen will.
#### Der Input
Festzulegen war, welche Informationen über einen Film wirklich relevant sind, um einen Vorschlag zu machen. Entschlossen haben wir uns als Gruppe auf folgende Informationen:
- Genres
- Schauspieler
- Regisseur
- Grundlegender Plot ( Keywords aus weiterem File )

Um diese Informationen gut verwenden zu können ist jedoch einiges an Datenaufbereitung nötig.

In [28]:
import pandas as pd

# movies = pd.read_csv("movie_data/movies_metadata.csv", low_memory=False)
# movie_actors = pd.read_csv("movie_data/credits.csv", low_memory=False)
# keywords = pd.read_csv("movie_data/keywords.csv")

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


# Datensätze verbinden

In [32]:
# join movies / credits and keywords on index, which has to be a string
movie_actors['id'] = movie_actors['id'].apply(str)
keywords['id'] = keywords['id'].apply(str)
merged = movies.set_index("id").join(movie_actors.set_index('id')).join(keywords.set_index('id'))

,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
id,,,,,,,,,,,,,,,,,,,,,
1997-08-20,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-09-29,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-01,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401840,False,NaN,0,[],NaN,tt3291632,es,School's out,Two high school kids mentored by a nightclub o...,0.207775,...,[],Released,NaN,School's out,False,0.0,0.0,NaN,NaN,NaN


# Datenaufbereitung

In [37]:
# remove elements without title or cast or keywords
merged = merged[merged["title"].notnull()]
merged = merged[merged["cast"].notnull()]
merged = merged[merged["keywords"].notnull()]

# delete not used columns
col_dels = ["adult", "belongs_to_collection", "homepage", "imdb_id", "poster_path", "production_countries",
            "tagline", "video", "original_title", "status", "original_language", "overview", "budget",
            "spoken_languages", "production_companies", 'release_date','vote_average','vote_count','revenue','runtime','popularity']
for col in col_dels:
    if col in merged.columns:
        del merged[col]

merged

# transform data
tf_genres = []
tf_cast = []
directors = []
tf_keywords = []
for index, row in merged.iterrows():
    # genres
    tf_genres_row = []
    genres = eval(row["genres"])
    for genre in genres:
        tf_genres_row.append(genre["name"])
    tf_genres.append(tf_genres_row)
    # keywords
    tf_key_row = []
    keys = eval(row["keywords"])
    for k in keys:
        tf_key_row.append(k["name"])
    tf_keywords.append(tf_key_row)
    # cast
    tf_cast_row = []
    casts = eval(row["cast"])
    for cast in casts:
        tf_cast_row.append(cast["name"])
    tf_cast.append(tf_cast_row)
    # director
    crew = eval(row["crew"])
    director = ""
    for person in crew:
        if person["job"]=="Director":
            director = person["name"]
            break
    directors.append(director)

            
# add data to table
merged["tf_genres"] = tf_genres
merged["tf_cast"] = tf_cast
merged["director"] = directors
merged["tf_keywords"] = tf_keywords

# change order of columns
cols = ['title','tf_genres','tf_cast','director','tf_keywords']
merged = merged[cols]

merged.head(10)

,title,tf_genres,tf_cast,director,tf_keywords
id,,,,,
100,"Lock, Stock and Two Smoking Barrels","[Comedy, Crime]","[Jason Flemyng, Dexter Fletcher, Nick Moran, J...",Guy Ritchie,"[ambush, alcohol, shotgun, tea, joint, machism..."
10000,La estrategia del caracol,"[Comedy, Drama]",[],Sergio Cabrera,"[roommate, pastor, squatter, anarchist, house,..."
10001,Young Einstein,"[Comedy, Science Fiction]","[Yahoo Serious, Odile Le Clezio, Peewee Wilson...",Yahoo Serious,"[atomic bomb, nobel prize, rock, albert einste..."
100010,Flight Command,"[Drama, War]","[Robert Taylor, Ruth Hussey, Walter Pidgeon, P...",Frank Borzage,"[pilot, navy]"
100017,Hounded,[Drama],"[Kostja Ullmann, Maren Kroymann, Moritz Grove,...",Angelina Maccarone,"[fetishism, masochism, submissive, older woman..."
10002,Mona Lisa,"[Drama, Crime, Romance]","[Bob Hoskins, Cathy Tyson, Michael Caine, Robb...",Neil Jordan,"[london england, prostitute, ex-detainee, chau..."
100024,Bloodwork,"[Horror, Thriller]","[Travis Van Winkle, Tricia Helfer, Eric Robert...",Eric Wostenberg,"[dangerous side effects, clinical trials, alle..."
10003,The Saint,"[Thriller, Action, Romance, Science Fiction, A...","[Val Kilmer, Elisabeth Shue, Rade Serbedzija, ...",Phillip Noyce,"[berlin, russia, gas, master thief, the saint]"
100032,The Great Los Angeles Earthquake,"[Drama, Action]","[Joanna Kerns, Dan Lauria, Bonnie Bartlett, Li...",Larry Elikann,[]


In [39]:
# namen konvertieren => kleinbuchstaben und Leerzeichen entfernen
def clean_names(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    elif isinstance(x, str):
        # director
        return str.lower(x.replace(" ", ""))

features = ['tf_cast', 'tf_keywords', 'director', 'tf_genres']

for feature in features:
    merged[feature] = merged[feature].apply(clean_names)

In [40]:
# Alle zu beachtenen Informationen werden hier aneinandergehängt und gespeichert
def create_summary(x):
    return ' '.join(x['tf_keywords']) + ' ' + ' '.join(x['tf_cast']) + ' ' + x['director'] + ' ' + ' '.join(x['tf_genres'])

merged["summary"] = merged.apply(create_summary, axis=1)

## Count Vectorizer
Durch das Nutzen des Count Vectorizers ist es uns möglich unsere erzeugten Text-Zusammenfassung in Verbindung zu setzen und eine Count-Matrix zu erstellen. Dies ist eine Matrix sogenannter Token Count, also einzelner Teilbegriffe innerhalb eines Strings. <br>
Anhand dieser Matrix können wir nun durch __fit_transform__ die erkannten Tokens zu einer Document-Term-Matrix machen, welche die Häufigkeit dieser Begriffe verarbeitet.

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(merged.head(2000)['summary'])

count_matrix

<2000x27693 sparse matrix of type '<class 'numpy.int64'>'
	with 45117 stored elements in Compressed Sparse Row format>

## Finale!!
Anhand der hier erstellten Matrix können wir die bereits beschriebene __Kosinus-Ähnlichkeit__ Methode anwenden, um die Ähnlichkeit verschiedener Filme zu realisieren und so Vorschläge zu machen.

In [43]:
# Kosinus-Ähnlichkeit matrix erstellen
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [45]:
# der index wird zurück gesetzt, so dass im folgenden dieser als spalte erreichbar ist
merged = merged.reset_index()
indices = pd.Series(merged.index, index=merged['title'])

In [50]:
# Für einen Film wird hier also geschaut, welche Filme die meisten Ähnlichkeiten haben und diese werden ausgegeben
def get_recommendations(title, cosine_sim):
    # Passenden Index zum Filmtitel holen
    idx = indices[title]

    # Aus der Kosinus-Ähnlichkeit für den Index die passenden Werte holen
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Nach größter Ähnlichkeit sortieren
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Auf Top-10 reduzieren
    sim_scores = sim_scores[1:11]

    # Indices für die Top-10
    movie_indices = [i[0] for i in sim_scores]

    # ID der passenden Indices zurückgeben (ID ist nicht mehr Index des DF)
    return merged['id'].iloc[movie_indices]

In [63]:
res = get_recommendations('Iron Man 2', cosine_sim)
# Beispielhaftes Ausgeben der Vorschläge mit Titel
for id in res.values:
    print(merged[merged["id"]==id]["title"].values[0])

Thor
Captain America: The Winter Soldier
Ant-Man
Captain America
Atom Man vs Superman
The Golden Bat
Spawn
Rules of Engagement
The Book of Fate
The incredible Paris Incident
